# Explain the concept of tidy data
Reference paper: [Tidy Data](https://vita.had.co.nz/papers/tidy-data.pdf)

Definition: 

In tidy data:
- Each variable forms a column
- Each observation forms a row
- Each type of observational unit forms a table

The five most common problems with messy datasets:
- Column headers are values, not variable names.
- Multiple variables are stored in one column.
- Variables are stored in both rows and columns.
- Multiple types of observational units are stored in the same table. 
- A single observational unit is stored in multiple tables.

Let's check them out!

# Transform the messy data into tidy data using Python
## Column headers are values, not variable names

For this particular example, we will recreate the original table from the original dataset of the Tidy Data paper

### Importing original dataset and extracting rows

In [1]:
# Original dataset http://pewforum.org/Datasets/Dataset-Download.aspx
# Sampple SPSS dataset from https://github.com/hadley/tidy-data/blob/master/data/pew.sav

import pandas as pd
import requests
import os.path

# pd.read_spss supports only file paths, so we need to download the file first
if os.path.exists('pew.sav'):
    print('File exists, skipping download')
else:
    response = requests.get('https://raw.github.com/hadley/tidy-data/master/data/pew.sav')
    response.raise_for_status()
    with open('pew.sav', 'wb') as file:
        file.write(response.content)
# To get the file directly, we need to access just the raw file, instead of the github page that serves the file. 
# Ref: https://stackoverflow.com/questions/14120502/how-to-download-and-write-a-file-from-github-using-requests

pew_df = pd.read_spss('pew.sav')
pew_df.head()

File exists, skipping download


,weight,psraid,int_date,lang,type,cregion,state,usr,usr1,form,...,q63,educ,income,regist,regicert,party,partyln,ideo,pvote04a,pvote04b
0,4.512821,10000001.0,50807.0,English,RDD,Northeast,Connecticut,Suburban,Suburban,Form A,...,"Yes, father born outside U.S.","Technical, trade, or vocational school AFTER h...","75 to under $100,000","Yes, registered",Absolutely certain,Republican,NaN,Moderate,Voted,Bush
1,2.102564,10000002.0,50807.0,English,RDD,Northeast,Maine,Rural,Rural,Form B,...,"No, both parents born in U.S.",High school graduate (Grade 12 or GED certific...,"20 to under $30,000","No, not registered",NaN,Republican,NaN,Conservative,Did not vote (includes too young to vote),NaN
2,1.282051,10000003.0,50807.0,English,RDD,Northeast,Maine,Rural,Rural,Form A,...,"No, both parents born in U.S.","College graduate (B.S., B.A., or other 4-year ...","30 to under $40,000","No, not registered",NaN,Independent,Democrat,Conservative,Did not vote (includes too young to vote),NaN
3,1.355323,10000004.0,50807.0,English,RDD,Northeast,Maine,Rural,Rural,Form B,...,"No, both parents born in U.S.","Some college, no 4-year degree (including asso...","Less than $10,000","No, not registered",NaN,Independent,Democrat,Moderate,Did not vote (includes too young to vote),NaN
4,1.589744,10000005.0,50807.0,English,RDD,Northeast,New York,Urban,Urban,Form A,...,"Yes, father born outside U.S.",Post-graduate training or professional schooli...,"50 to under $75,000","Yes, registered",Absolutely certain,Independent,Democrat,Moderate,Voted,Other candidate


In [2]:
# We extract these particular columns to create the table
# q16: religious beliefs?
# reltrad: religious tradition
# income: income in USD$
religion = pew_df[['q16', 'reltrad', 'income']]
religion.head()

,q16,reltrad,income
0,Protestant,Evangelical Protestant Churches,"75 to under $100,000"
1,Protestant,Mainline Protestant Churches,"20 to under $30,000"
2,Protestant,Mainline Protestant Churches,"30 to under $40,000"
3,Nothing in particular,Unaffiliated,"Less than $10,000"
4,Jewish (Judaism),Jewish,"50 to under $75,000"


In [3]:
religion['q16'].value_counts()

 Protestant                               18353
 Roman Catholic                            8047
 Nothing in particular                     3707
 Agnostic (not sure if there is a God)      826
 Jewish (Judaism)                           682
                                          ...  
 Satanism (VOL)                               1
 Unification Church (Moonies) (VOL)           1
 Maronite Catholic (VOL)                      1
 Animism (VOL)                                1
 Old Catholic (VOL)                           1
Name: q16, Length: 65, dtype: int64

### Data cleaning

In [4]:
# TODO Original versions return SettingWithCopyWarning messages
# Looking for ways to replace them for .loc versions
# All these transformation appear on original Tidy Data paper
religion['reltrad'] = religion['reltrad'].str.replace(' Protestant', ' Prot') 
religion['reltrad'] = religion['reltrad'].str.replace(' Churches', '')

/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_16398/3389848522.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['reltrad'] = religion['reltrad'].str.replace(' Protestant', ' Prot')
/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_16398/3389848522.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['reltrad'] = religion['reltrad'].str.replace(' Churches', '')


#### Warning
<div class="alert alert-block alert-warning">
<b>df.replace()</b>: replaces the content of a whole cell. It supports inplace=True argument. <br>
<b>df.str.replace()</b>: replaces fragments of strings in a cell. It doesn't support inplace=True argument.
</div>

In [5]:
# Assign a value for reltrad based on the q16 contents
# R versions: religion$reltrad[religion$q16 == " Atheist (do not believe in God) "] <- "Atheist"
# R versions: religion$reltrad[religion$q16 == " Agnostic (not sure if there is a God) "] <- "Agnostic"

# Note that in original example, there are leading and trailing whitespaces. Somehow ending whitespace was trimmed on import
# TODO Report a bug to pandas?

# So we match without using the final space!

# A direct translation of R code would be:
# religion.loc[religion['q16'].str.match(' Atheist (do not believe in God)'), 'reltrad'] = 'Atheist'
# religion.loc[religion['q16'].str.match(' Agnostic (not sure if there is a God)'), 'reltrad'] = 'Agnostic'
# However, in Python we need to escape parenthesis, or it will return zero matches.
# religion.loc[religion['q16'].str.match(' Atheist \(do not believe in God\)'), 'reltrad'] = 'Atheist'
# religion.loc[religion['q16'].str.match(' Agnostic \(not sure if there is a God\)'), 'reltrad'] = 'Agnostic'
# To simplify the string matching, we will use str.contains instead of match

religion.loc[religion['q16'].str.contains('Atheist'), 'reltrad'] = 'Atheist'
religion.loc[religion['q16'].str.contains('Agnostic'), 'reltrad'] = 'Agnostic'

# Alternative, traditional string comparison
# religion.loc[religion['q16'] == (' Agnostic (not sure if there is a God)'), 'reltrad'] = 'Agnostic'


/Users/cesargarciasaez/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


#### Testing cells for matching embedded parenthesis in strings

In [6]:
num_prots = sum(religion['q16'].str.match(' Protestant'))
print(num_prots)

18353


In [7]:
num_agn = sum(religion['q16'].str.match(' Agnostic \(not sure if there is a God\)'))
print(num_agn)

826


In [8]:
# TODO Not working with this code
# religion['reltrad'].str.replace(r'\([^)]*\)', '', regex=True)
# religion['reltrad'].str.strip()

# Remove (expressions inside parenthesis) in reltrad
religion['reltrad'] = religion['reltrad'].str.replace(r'\([^)]*\)', '', regex=True)

# Trim spaces in reltrad: R version religion$reltrad <- str_trim(religion$reltrad)
religion['reltrad'] = religion['reltrad'].str.strip()

/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_16398/3148514731.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['reltrad'] = religion['reltrad'].str.replace(r'\([^)]*\)', '', regex=True)
/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_16398/3148514731.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['reltrad'] = religion['reltrad'].str.strip()


In [9]:
# Replace income values for shorter versions. I will use using map in this particular case. It seems to be more efficient! 
# Ref: https://stackoverflow.com/questions/42012339/using-replace-efficiently-in-pandas
# Note: income is already categorical data, so there is no need for binning actual incomes. These are already bins!

income_to_shorter_description = {
  "Less than $10,000": "<$10k", 
  "10 to under $20,000": "$10-20k", 
  "20 to under $30,000": "$20-30k", 
  "30 to under $40,000": "$30-40k", 
  "40 to under $50,000": "$40-50k", 
  "50 to under $75,000": "$50-75k",
  "75 to under $100,000": "$75-100k", 
  "100 to under $150,000": "$100-150k", 
  "$150,000 or more": ">150k", 
  "Don't know/Refused (VOL)": "Don't know/refused"
}

religion['income'] = religion['income'].map(income_to_shorter_description)
religion.head()

/var/folders/p7/xh10c8b91k92wwpxwr69f38r0000gn/T/ipykernel_16398/2322833425.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  religion['income'] = religion['income'].map(income_to_shorter_description)


,q16,reltrad,income
0,Protestant,Evangelical Prot,$75-100k
1,Protestant,Mainline Prot,$20-30k
2,Protestant,Mainline Prot,$30-40k
3,Nothing in particular,Unaffiliated,<$10k
4,Jewish (Judaism),Jewish,$50-75k


### Reconstructing original table

In [10]:
# Now it's time to sort how many people are in each income group for each reltrad
# We will keep the original variable names here

counts = religion.groupby(['reltrad', 'income']).size()
# size() returns a Series with multi-index
# count() returns a Dataframe with multi-index and a column

In [11]:
# Reltrad appears as religion in the Tidy Data paper, so we have to rename the index
counts.index.set_names({'reltrad': 'religion'}, inplace=True)
counts.sort_index()
counts.index

MultiIndex([(             'Agnostic',   '$10-20k'),
            (             'Agnostic', '$100-150k'),
            (             'Agnostic',   '$20-30k'),
            (             'Agnostic',   '$30-40k'),
            (             'Agnostic',   '$40-50k'),
            (             'Agnostic',   '$50-75k'),
            (             'Agnostic',  '$75-100k'),
            (             'Agnostic',     '<$10k'),
            (             'Agnostic',     '>150k'),
            (              'Atheist',   '$10-20k'),
            ...
            ('Other World Religions',     '>150k'),
            (         'Unaffiliated',   '$10-20k'),
            (         'Unaffiliated', '$100-150k'),
            (         'Unaffiliated',   '$20-30k'),
            (         'Unaffiliated',   '$30-40k'),
            (         'Unaffiliated',   '$40-50k'),
            (         'Unaffiliated',   '$50-75k'),
            (         'Unaffiliated',  '$75-100k'),
            (         'Unaffiliated',     '<$10k

In [12]:
# This is our final destination, where we have been able to reproduce the table in the original paper.
# At the same time, this is the first step to start tidyng the data!
counts.unstack().head(10)

income,$10-20k,$100-150k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,<$10k,>150k
religion,,,,,,,,,
Agnostic,34,109,60,81,76,137,122,27,84
Atheist,27,59,37,52,35,70,73,12,74
Buddhist,21,39,30,34,33,58,62,27,53
Catholic,617,792,732,670,638,1116,949,418,633
Don’t know/refused,14,17,15,11,10,35,21,15,18
Evangelical Prot,869,723,1064,982,881,1486,949,575,414
Hindu,9,48,7,9,11,34,47,1,54
Historically Black Prot,244,81,236,238,197,223,131,228,78
Jehovah's Witness,27,11,24,24,21,30,15,20,6


### How would I do it if I had to start from scratch?
I would change the order of the cleaning operations to simplify the process. For example:

#### I would strip leading and trailing whitespaces in all the columns first

All reltrad rows start by a white space. In the original R file, they also ended by a white space. In the version I read, the ending white space is missing.

He matches the string with spaces, and then later removes all spaces. I think it is more clear and less error prone to strip white spaces first and then do the string matching/replacement.

White spaces can haunt you in so many unexpected places!
```python
MultiIndex([(' Buddhist',   '$10-20k'),
            (' Buddhist', '$100-150k'),
            (' Buddhist',   '$20-30k'),
            (' Buddhist',   '$30-40k'),
            (' Buddhist',   '$40-50k'),
            (' Buddhist',   '$50-75k'),
            (' Buddhist',  '$75-100k'),
            (' Buddhist',     '<$10k'),
            (' Buddhist',     '>150k'),
            (' Catholic',   '$10-20k'),
            ...
            ( 'Agnostic',     '>150k'),
            (  'Atheist',   '$10-20k'),
```

Did you see the whitespace before Buddhist scrambles any sort?

#### I would remove information under parenthesis earlier
If there information between parenthesis is not needed and doesn't offer any additional insights, it could make sense to remove them as early as possible, to make the output more readable and visible.

#### I would start creating softer string matches and then do stronger ones
I would start using str.contains() and then test with str.match(). I have spent too many hours figuring out why I could not match an expression that included whitespaces and parenthesis!

### Further research
- [ ] TODO I need to make a table with operations that require inplace=True, and those that will change the dataframe inplace by default.
- [ ] TODO I need to remove all the SettingWithCopyWarning errors. Example: religion['reltrad'] = religion['reltrad'].str.strip() (religion['reltrad'].str.strip() would not modify the original values unless this assignation is specified)
- [ ] TODO In the multi-index above, the categories are not sorted properly by income. Right now it is sorted like: $10k, $100k, $20k, [...], < $10k, > $10k.
- [ ] TODO The "Don't know / refused" category is missing from the output. In the paper, they mention that they have filtered some rows, but we haven't done that in our code AFAIK.

### Melting the dataframe to produce tidy data output
In order to produce tidy data format, we have to make sure that every row is a single observation, and that each column is a variable. These are called **colvars**.

When we melt, we need to pass the list of columns that are already variables, plus a list of other columns that will be converted into variables.

In this particular case, we have just a colvar, called *religion*. As in the original paper, we will rename the value column to *freq*. Variable column is already called *income*.

Please also note that most of the following steps are just unnecesary, as the *counts* dataframe created above as a multi-index series could be already be considered tidy data. We are doing the next steps for the sake of completeness.

In [13]:
# counts.reset_index(inplace=True) returns Cannot reset_index inplace on a Series to create a DataFrame 
rel_income_df =  counts.reset_index()
rel_income_df.head(10)


,religion,income,0
0,Agnostic,$10-20k,34
1,Agnostic,$100-150k,109
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Agnostic,$75-100k,122
7,Agnostic,<$10k,27
8,Agnostic,>150k,84
9,Atheist,$10-20k,27


In [14]:
# As we extracted our data from a series of int64, all data types were lost along. We need to set the data types again.
rel_income_df['religion'] = rel_income_df['religion'].astype('category')
rel_income_df['income'] = rel_income_df['income'].astype('category')
rel_income_df.dtypes

religion    category
income      category
0              int64
dtype: object

In [15]:
# Then we rename to 0 column to freq
rel_income_df.rename(columns={0: 'freq'}, inplace=True)
rel_income_df.head()

,religion,income,freq
0,Agnostic,$10-20k,34
1,Agnostic,$100-150k,109
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76


In [16]:
# Now we set the correct order for the income category
income_order_list = ['<$10k', '$10-20k', '$20-30k',	'$30-40k','$40-50k', '$50-75k', '$75-100k', '$100-150k', '>150k', "Don't know / refused"]
rel_income_df['income'] = rel_income_df['income'].cat.set_categories(income_order_list, ordered=True)
# Without the following line, category is ordered but the dataframe is not. So we need to sort values inplace.
rel_income_df.sort_values(['religion','income'], inplace=True)
rel_income_df.head(10)


,religion,income,freq
7,Agnostic,<$10k,27
0,Agnostic,$10-20k,34
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Agnostic,$75-100k,122
1,Agnostic,$100-150k,109
8,Agnostic,>150k,84
16,Atheist,<$10k,12


In [17]:
# To make it event tidier, let's reset the index to the current order of rows.
rel_income_df.reset_index(drop=True, inplace=True)
rel_income_df.head(10)

,religion,income,freq
0,Agnostic,<$10k,27
1,Agnostic,$10-20k,34
2,Agnostic,$20-30k,60
3,Agnostic,$30-40k,81
4,Agnostic,$40-50k,76
5,Agnostic,$50-75k,137
6,Agnostic,$75-100k,122
7,Agnostic,$100-150k,109
8,Agnostic,>150k,84
9,Atheist,<$10k,12


## Multiple variables are stored in one column

## Variables are stored in both rows and columns

## Multiple types of observarional units are stored in the same table

## A single observational unit is stored in multiple tables

# Fundamental operations
There are three fundamental operations:
- Pivot
- Stack / Unstack
- Melt

Reference: https://pandas.pydata.org/docs/user_guide/reshaping.html

Reference: https://wesmckinney.com/book/data-wrangling.html#prep_reshape